In [ ]:
class MantridCli(object):
    """Command line interface to Mantrid"""

    def __init__(self, base_url=None):
        #self.client = MantridClient(base_url)

    @classmethod
    def main(cls):
        cli = cls("http://localhost:8042")
        cli.run(sys.argv)

    @property
    def action_names(self):
        for method_name in dir(self):
            if method_name.startswith("action_") \
               and method_name != "action_names":
                yield method_name[7:]
        
    def run(self, argv):
        # Work out what action we're doing
        try:
            action = argv[1]
        except IndexError:
            sys.stderr.write(
                "Please provide an action (%s).\n" % (
                    ", ".join(self.action_names),
                )
            )
            sys.exit(1)
        if action not in list(self.action_names):
            sys.stderr.write(
                "Action %s does not exist.\n" % (
                    action,
                )
            )
            sys.exit(1)
        # Run it
        getattr(self, "action_%s" % action)(*argv[2:])
    
    def action_list(self):
        "Lists all hosts on the LB"
        format = "%-35s %-25s %-8s"
        print format % ("HOST", "ACTION", "SUBDOMS")
        for host, details in sorted(self.client.get_all().items()):
            if details[0] in ("proxy", "mirror"):
                action = "%s<%s>" % (
                    details[0],
                    ",".join(
                        "%s:%s" % (host, port)
                        for host, port in details[1]['backends']
                    )
                )
            elif details[0] == "static":
                action = "%s<%s>" % (
                    details[0],
                    details[1]['type'],
                )
            elif details[0] == "redirect":
                action = "%s<%s>" % (
                    details[0],
                    details[1]['redirect_to'],
                )
            elif details[0] == "empty":
                action = "%s<%s>" % (
                    details[0],
                    details[1]['code'],
                )
            else:
                action = details[0]
            print format % (host, action, details[2])
    
    def action_set(self, hostname=None, action=None, subdoms=None, *args):
        "Adds a hostname to the LB, or alters an existing one"
        usage = "set <hostname> <action> <subdoms> [option=value, ...]"
        if hostname is None:
            sys.stderr.write("You must supply a hostname.\n")
            sys.stderr.write("Usage: %s\n" % usage)
            sys.exit(1)
        if action is None:
            sys.stderr.write("You must supply an action.\n")
            sys.stderr.write("Usage: %s\n" % usage)
            sys.exit(1)
        if subdoms is None or subdoms.lower() not in ("true", "false"):
            sys.stderr.write("You must supply True or False for the subdomains flag.\n")
            sys.stderr.write("Usage: %s\n" % usage)
            sys.exit(1)
        # Grab options
        options = {}
        for arg in args:
            if "=" not in arg:
                sys.stderr.write("%s is not a valid option (no =)\n" % (
                    arg
                ))
                sys.exit(1)
            key, value = arg.split("=", 1)
            options[key] = value
        # Sanity-check options
        if action in ("proxy, mirror") and "backends" not in options:
            sys.stderr.write("The %s action requires a backends option.\n" % action)
            sys.exit(1)
        if action == "static" and "type" not in options:
            sys.stderr.write("The %s action requires a type option.\n" % action)
            sys.exit(1)
        if action == "redirect" and "redirect_to" not in options:
            sys.stderr.write("The %s action requires a redirect_to option.\n" % action)
            sys.exit(1)
        if action == "empty" and "code" not in options:
            sys.stderr.write("The %s action requires a code option.\n" % action)
            sys.exit(1)
        # Expand some options from text to datastructure
        if "backends" in options:
            options['backends'] = [
                (lambda x: (x[0], int(x[1])))(bit.split(":", 1))
                for bit in options['backends'].split(",")
            ]
        # Set!
        self.client.set(
            hostname,
            [action, options, subdoms.lower() == "true"]
        )
    
    def action_delete(self, hostname):
        "Deletes the hostname from the LB."
        self.client.delete(
            hostname,
        )
    
    def action_stats(self, hostname=None):
        "Shows stats (possibly limited by hostname)"
        format = "%-35s %-11s %-11s %-11s %-11s"
        print format % ("HOST", "OPEN", "COMPLETED", "BYTES IN", "BYTES OUT")
        for host, details in sorted(self.client.stats(hostname).items()):
            print format % (
                host,
                details.get("open_requests", 0),
                details.get("completed_requests", 0),
                details.get("bytes_received", 0),
                details.get("bytes_sent", 0),
            )

In [ ]:
stats_dict = self.stats.setdefault(action.matched_host, {})

In [58]:
%%file remote_api_depend.py
import random
from Arthur.core.apiserver.client import Client
from Arthur.core.entities.base.bs_log import Log
from .redis_api import storage,CountApi

class data_api(object):
    def __init__(self,redis_host='127.0.0.1',
                 redis_port=6379,redis_db=5,
                 directory="."):

        self.redis_host=redis_host
        self.redis_port=redis_port
        self.redis_db=redis_db
    def countvisit(self,prefix='arthurService:'):
        store=storage('redis', flush_db=False, host=self.redis_host, port=self.redis_port, db= self.redis_db)
        return CountApi(store,prefix)

class ArthurflyError(Exception):
    pass
def RemoteFunction(client, func_name):
    def wrapped(*args, **kwargs):
        if args:
            raise ArthurflyError('Arthur service functions only accept named arguments')
        return client.call_func(func_name, **kwargs)
    wrapped.__name__ = func_name
    wrapped.__qualname__ = func_name
    wrapped.__doc__ = client.get_doc(func_name)
    return wrapped


class RemoteAPIDriver(object):
    """API for Accessing Arthur Services
    Parameters
    ----------
    remote ip: str
        model service remote path(host/ports)
    service function: str
        predict function
    input data: json
        post data to get model predict result
    api_key/token : str
        credentials to access remote
    Attributes
    ----------
    Methods
    -------
    post_data(input_data)
        post single data point to Arthur scoped by model
    get_data(filter)
        get data information from Arthur scoped by filter
    update_actual(id, actual)
        update the data id with actual values (y_hat)
    get_deployment_info()
        returns deployment info from Arthur deployed models
    """
    def __init__(self, function='def',host='0.0.0.0',port=8001,token=""):
        self.func=''
        self.request_cnt = data_api().countvisit()
    def get_service_key(self, service_name=None, host=None,
                              port=None, address=None,service=None):
        name = service_name or service.name
        #port = port or service.port
        #host = host or service.host
        #k = 'completed_requests:%s:%s:%s' % (name,host,port) or 'completed_requests:%s:%s' % (name,address)
        k = 'completed_requests:%s:%s' % (name,address)
        #print 'get_service_key: %s' % k
        Log.info('get_service_key: %s' % k)
        return k
    @staticmethod    
    def resolve_host(address, protocol="http"):
        # Check for an exact or any subdomain matches
        if isinstance(address,list):
            if len(address)>0:
                backend = random.choice(address)
                return "%s://" % protocol + backend
            else:
                raise Exception('please give remote address')
        else:
            return "%s://" % protocol + address 
    def get_completed_requests(self,function,address):
        completed_requests_key=self.get_service_key(service_name=function, address=address)
        return self.request_cnt[completed_requests_key]

    def handle(self, address, function,input_data,token=''):
        """
        Handles an incoming HTTP connection.
        """
        open_requests=20
        completed_requests=10
        response = {"status_code": 200}
        try:
            backend= self.resolve_host(address)
            client = Client(backend,auth_token=token)
            fn = RemoteFunction(client,function)
            predict_out = fn(**input_data)
            response = {"status_code": 200, "body": predict_out}
        except:
            #raise Exception('there is some error when call function %s '%function)
            response['status_code'] = 500
        finally:
            completed_requests_key=self.get_service_key(service_name=function, address=address)
            #print completed_requests_key
            self.request_cnt[completed_requests_key]=1
            #completed_requests+=1
            #print 'completed_requests',completed_requests
            #stats_dict['open_requests'] -= 1
            #stats_dict['completed_requests'] = stats_dict.get('completed_requests', 0) + 1
            #stats_dict['bytes_sent'] = stats_dict.get('bytes_sent', 0) + sock.bytes_sent
            #stats_dict['bytes_received'] = stats_dict.get('bytes_received', 0) + sock.bytes_received
        return response
test=RemoteAPI() 
test.handle('127.0.0.1:5006', 'fib',{"n":20},token='')

Overwriting remote_api_depend.py


In [22]:
def resolve_host(remote_path, protocol="http"):
    # Check for an exact or any subdomain matches
    if isinstance(remote_path,list) :
        if len(remote_path)>0:
            backend = random.choice(remote_path)
            return "%s://" % protocol + backend
        else:
            raise Exception('please give remote address')
        
    else:
        return "%s://" % protocol + remote_path
            

resolve_host([])

Exception: please give remote address

In [14]:
if isinstance('fg',str):
    print 'f'

f


In [19]:
import random
backends=['d']
tuple(random.choice(backends))

('d',)

In [3]:
random.choice([1, 2, 3, 5, 9])

2

In [34]:
%%file remote_api_independ.py
import random
from Arthur.core.apiserver.client import Client

class ArthurflyError(Exception):
    pass
def RemoteFunction(client, func_name):
    def wrapped(*args, **kwargs):
        if args:
            raise ArthurflyError('Arthur service functions only accept named arguments')
        return client.call_func(func_name, **kwargs)
    wrapped.__name__ = func_name
    wrapped.__qualname__ = func_name
    wrapped.__doc__ = client.get_doc(func_name)
    return wrapped


class RemoteAPI():
    """API for Accessing Arthur Services
    Parameters
    ----------
    remote ip: str
        model service remote path(host/ports)
    service function: str
        predict function
    input data: json
        post data to get model predict result
    api_key/token : str
        credentials to access remote
    Attributes
    ----------
    Methods
    -------
    post_data(input_data)
        post single data point to Arthur scoped by model
    get_data(filter)
        get data information from Arthur scoped by filter
    update_actual(id, actual)
        update the data id with actual values (y_hat)
    get_deployment_info()
        returns deployment info from Arthur deployed models
    """
    def __init__(self, function='def',host='0.0.0.0',port=8001,token=""):
        self.func=''
    @staticmethod    
    def resolve_host(address, protocol="http"):
        # Check for an exact or any subdomain matches
        if isinstance(address,list):
            if len(address)>0:
                backend = random.choice(address)
                return "%s://" % protocol + backend
            else:
                raise Exception('please give remote address')
        else:
            return "%s://" % protocol + address 

    def handle(self, address, function,input_data,token=''):
        """
        Handles an incoming HTTP connection.
        """
        open_requests=20
        completed_requests=10
        try:
            backend= self.resolve_host(address)
            client = Client(backend,auth_token=token)
            fn = RemoteFunction(client,function)
            predict_out = fn(**input_data)
            response = {"status_code": 200, "body": predict_out}
            return response
        except:
            raise Exception('there is some error when call function %s '%function)
        finally:
            completed_requests+=1
            print 'completed_requests',completed_requests
            #stats_dict['open_requests'] -= 1
            #stats_dict['completed_requests'] = stats_dict.get('completed_requests', 0) + 1
            #stats_dict['bytes_sent'] = stats_dict.get('bytes_sent', 0) + sock.bytes_sent
            #stats_dict['bytes_received'] = stats_dict.get('bytes_received', 0) + sock.bytes_received
test=RemoteAPI() 
test.handle('127.0.0.1:5006', 'fib',{"n":20},token='')

Writing remote_api_independ.py


In [43]:
%%file redis_api.py
# -*- coding: UTF-8 -*- 
import abc
import redis

__all__ = ['storage']


def storage(storage_type, **kwargs):
    if not storage_type:
        return DictStorage()
    elif storage_type == 'redis':
        return RedisStorage(**kwargs)
    else:
        raise ValueError('storage_type not supported.')


class Storage(object):
    @abc.abstractmethod
    def __init__(self, flush_db=False, **kwargs):
        """
        Initializes an object to store values. If the storage needs a database connection, it also
        connects to the database.
        """
        return

    @abc.abstractmethod
    def __del__(self):
        """
        Deletes in-memory data structures
        """
        return

    @abc.abstractmethod
    def __getitem__(self, item):
        """
        Looks for the `item` and possibly returns its value. None is returned if `item` is not present.
        """
        return

    @abc.abstractmethod
    def __setitem__(self, key, value):
        """
        Stores `value` as the value of item  named `key`. If `key` is not present it is added to the storage.
        If `key` contained an old value, the old value is overwritten.
        """
        return

    def incrby(self, key, incr):
        if key not in self:
            self[key] = 0
        self[key] = int(self[key]) + incr
        return int(self[key])


class RedisStorage(Storage):
    def __init__(self, flush_db=False, **kwargs):
        r = redis.StrictRedis(**kwargs)
        if flush_db:
            r.flushdb()
        self._r = r

    def __del__(self):
        del self._r

    def __getitem__(self, key):
        val = self._r.get(key)
        try:
            return int(val)
        except ValueError:
            return val

    def __setitem__(self, key, value):
        self._r.set(key, value)

    def __contains__(self, key):
        return True if self._r.get(key) else False

    def smembers(self, key):
        """
        Get all the members in a set.
        """
        res = self._r.smembers(key)
        return set([int(el) if el.isdigit() else el for el in res])  # possibly convert values to integers

    def sadd(self, key, value):
        """
        Insert a `value` into a set.
        """
        self._r.sadd(key, value)

    def sclear(self, key):
        """
        Clear the contents of a set
        """
        self._r.delete(key)
    def _srem(self, set_name,value):
        """
        Clear the contents of a set
        """
        self._r.srem(set_name,value)
    def _scard(self, set_name):
        """
        lookup the contents of a set
        """
        return self._r.scard(set_name)
    def keys(self):
        return self._r.keys()


class DictStorage(Storage):
    def __init__(self):
        self._items = dict()

    def __del__(self):
        self._items.clear()
        del self._items

    def __getitem__(self, key):
        return self._items[key] if key in self._items else None

    def __setitem__(self, key, value):
        self._items[key] = value

    def __contains__(self, key):
        return key in self._items

    def smembers(self, key):
        """
        Get all the members in a set.
        """
        return self._items[key] if key in self._items else set()

    def sadd(self, key, value):
        """
        Insert a `value` into a set.
        """
        if not key in self._items:
            self._items[key] = set([value])
        else:
            self._items[key].update([value])

    def sclear(self, key):
        """
        Clear the contents of a set
        """
        self._items[key].clear()

    def keys(self):
        return self._items.keys()
    
def prepender(func):
    """
    Prepends some text to the *second* argument with which function `func` has been called.
    The *first* argument is a reference (`self`) to an instance of class Terms or one of its sub classes.
    `self` is accessed to look for a prefix `_prefix` to append to the second argument.
    This function is meant to be used as decorator.
    """
    def prepend(self, *args):
        prepended = self._prefix + args[0]
        return func(self, prepended, *args[1:])
    return prepend

class Terms(object):
    def __init__(self, store=None):
        self._items = store
    @property
    def terms(self):
        """
        Returns all the terms we have stored without their prefix
        """
        return [k[len(self._prefix):] for k in self._items.keys() if k.startswith(self._prefix)]


class CountApi(Terms):
    #_prefix = pre#'t:'  # this prefix stands for `term:`

    def __init__(self, store, pre='tre'):
        self._prefix = pre
        _prefix = self._prefix
        super(CountApi, self).__init__(store)
    @prepender
    def __setitem__(self, word, count):
        """
        Adds the `word` to the original terms. The number of occurrences is specified in `count`.
        """
        self._items.incrby(word, count)

    @prepender
    def __getitem__(self, word):
        """
        Returns the number of occurrences of `word` in the corpus or 0 if it wasn't present.
        """
        return self._items[word] if word in self._items else 0

class SetApi(Terms):
    #_prefix = pre#'t:'  # this prefix stands for `term:`

    def __init__(self, store, pre='tre'):
        self._prefix = pre
        _prefix = self._prefix
        super(SetApi, self).__init__(store)
    @prepender
    def __setitem__(self, key, val):
        """
        Adds the `word` to the original terms. The number of occurrences is specified in `count`.
        """
        self._items[key]=val

    @prepender
    def __getitem__(self, word):
        """
        Returns the number of occurrences of `word` in the corpus or 0 if it wasn't present.
        """
        return self._items[word] if word in self._items else 0
class MemApi(Terms):
    #_prefix = 's:'  # this prefix stands for `suggestion:`

    def __init__(self, store,pre='tre'):
        self._prefix = pre
        _prefix = self._prefix
        super(MemApi, self).__init__(store)

    @prepender
    def __setitem__(self, delete, suggestion):
        """
        Adds the `delete` term with the corresponding `suggestion`.
        """
        self._items.sadd(delete, suggestion)
    @prepender
    def __delitem__(self, delete):
        """
        Adds the `delete` term with the corresponding `suggestion`.
        """
        self._items.sclear(delete)
    @prepender
    def __getitem__(self, word):
        return self._items.smembers(word)
    @prepender
    def srem(self, set_name,value):
        """
        removes value from sadd setkey  set_name,value
        """
        self._items._srem(set_name,value)
    @prepender
    def scard(self,value):
        return self._items._scard(value)
        
    


"""
###test:
redis_host = 'localhost'
redis_port = 6379
redis_db = 5
def redis_storage():
    return storage('redis', flush_db=False, host=redis_host, port=redis_port, db=redis_db)
store=redis_storage()
oi = OriginalTerms(store,'ddd')
oi['foo']=1
r = redis.StrictRedis(db=redis_db)
print r.get('t:foo')
"""

Writing redis_api.py


In [36]:
from redis_api import storage,CountApi
class data_api(object):
    def __init__(self,redis_host='127.0.0.1',
                 redis_port=6379,redis_db=5,
                 directory="."):

        self.redis_host=redis_host
        self.redis_port=redis_port
        self.redis_db=redis_db
    def countvisit(self,prefix='arthurService:'):
        store=storage('redis', flush_db=False, host=self.redis_host, port=self.redis_port, db= self.redis_db)
        return CountApi(store,prefix)

In [37]:
x=data_api().countvisit()

In [41]:
x['cont']=1

In [42]:
x['cont']

2